In [2]:
#import data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
#suppress warnings
import warnings
warnings.filterwarnings('ignore')
#import /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/Ozturk 2022/Data_SupplementaryFigure_1h.txt
df = pd.read_csv('/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/Ozturk 2022/Data_Figure_1b.txt', sep='\t')
#import the mapping file /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/external data/gene_IDs_names_products.tsv
map = pd.read_csv('/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/external data/gene_IDs_names_products.tsv', sep='\t',header=None)


In [3]:
map = pd.read_csv('/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/external data/gene_IDs_names_products.tsv', sep='\t',header=None)

In [4]:
#rename columns without spaces before the names
df.columns = df.columns.str.replace(' ', '')
#create a column with name deletion and fill with np.nan. If strainType is "3,308 knockout strains" then fill with what is after - in Strain
df['deletion'] = "WT"
df.loc[df['strainType'] == '3,308 knockout strains', 'deletion'] = df['Strain'].str.split('-').str[1]

In [17]:
#"SPCC594.01" in proteingroup?
sum(df['proteinGroup'].str.contains('SPCC594.01'))


3769

In [5]:
#dictionary of interesting deletions
deletions_positive= {
"SPBC317.01" : "mbx2",
"SPCC1223.13" : "cbf12",

}
deletions_negative= {
    "SPBC15D4.02":"gsf1",
    "SPCC736.08":"cbf11",
    "SPBC106.19":"ppr10",
    "SPBC19G7.07c":"ppr3",
    "SPAC8C9.06c":"ppr4",
    "SPCC11E10.04":"ppr6",
    "SPAC1D4.11c":"lkh1",
    "SPBC21B10.13c":"yox1",
    "SPBC354.05c":"sre2"
}


genes_positive = list(deletions_positive.keys())
genes_negative = list(deletions_negative.keys())


In [6]:
def get_proteome_hits(df, map,genes):
    #loop through genes. find rows where deletion entry equals the value. Add this to a temporary dataframe. Concatenate it with the other dataframes.
    df_temp = pd.DataFrame()
    #also create a complementary temp df which contains entries where deletion is WT
    df_temp2 = df.loc[df['deletion'] == "WT"]
    for gene in genes:
        df_temp = pd.concat([df_temp, df.loc[df['deletion'] == gene]])
       
        #
    #for each proteinGroup, do a t-test between entries in df_temp and df_temp2. Add to a new dataframe
    df_res = pd.DataFrame(columns=['proteinGroup', 'p-value',"t_stat"])
    for i,proteinGroup in enumerate(df_temp['proteinGroup'].unique()):
        #print progression
        all =len(df_temp['proteinGroup'].unique())
        print(i/all)
        #get the entries for the proteinGroup
        df_temp3 = df_temp.loc[df_temp['proteinGroup'] == proteinGroup]
        
        df_temp4 = df_temp2.loc[df_temp2['proteinGroup'] == proteinGroup]
        #do a t-test
        ttest = st.ttest_ind(df_temp3['log2(intensity)'], df_temp4['log2(intensity)'])
        
        #add proteinGroup and p-value to df_res
        df_res = df_res.append({'proteinGroup' : proteinGroup, 'p-value' : ttest[1], "t_stat" : ttest[0]  }, ignore_index=True)
    #sort by p-value
    df_res = df_res.sort_values(by=['p-value'])
    #reset index
    df_res = df_res.reset_index(drop=True)
    # create a column gene that is the copy of proteinGroup
    df_res['gene'] = df_res['proteinGroup']
    df_res['role'] = np.nan
    # #iterate through df_res proteinGroups and find the corresponding entry in map[0]. Find the corresponding map[2] and add to a new column in df_res
    for i,proteinGroup in enumerate(df_res['proteinGroup']):
        
        #find the corresponding entry in map[0]
        #if proteinGroup in map[0].values and it isnt NaN
        if (proteinGroup in map[0].values) and (str(map[2][map[0] == proteinGroup].values[0]) != "nan"):
            #find the corresponding map[2] and add to a new column in df_res
            
            df_res.loc[i, 'gene'] = map[2][map[0] == proteinGroup].values[0]
        #else it is just itself
        else:
            df_res.loc[i, 'gene'] = proteinGroup

      
    return df_res

  #add information from column 4 to the df
        
# df_res.loc[i, 'role'] = map[4][map[0] == proteinGroup].values[0]


In [7]:
#unique deletion how many
df['deletion'].nunique()

3309

In [11]:
for i in df_res_neg.head(50):
    #print if i is in df gene column
    print(i in df["deletion"].values)

False
True
True
True
True
True
True
True
True


In [21]:
df_res_neg.head(10)["gene"]

0           duc1
1           atd1
2           rmt3
3           adh1
4          rps21
5           irs1
6    SPBC16E9.15
7           ura1
8           elo2
9           pet2
Name: gene, dtype: object

In [22]:
#take top 10 hits from df_res_neg, find the corresponding values from deletion strains and plot them
#also plot the WT values
import iqplot
import bokeh
from bokeh.io import output_notebook, show

for i in df_res_neg.head(10)["proteinGroup"]:
    #get the WT values
    df_temp = df.loc[(df['deletion'] == "WT") & (df['proteinGroup'] == i)]
    #get the values for the deletion strains taht are in genes_negative
    df_temp2 = df.loc[(df['deletion'].isin(genes_negative)) & (df['proteinGroup'] == i)]
    #concatenate the two dataframes and add a column with the gene name
    #if first time through, create a new dataframe
    if i == df_res_neg.head(10)["proteinGroup"].iloc[0]:
        df_temp3 = pd.concat([df_temp, df_temp2])
        df_temp3['POI'] = i
    else :
        df_temp4 = pd.concat([df_temp, df_temp2])
        df_temp4['POI'] = i
        df_temp3 = pd.concat([df_temp3, df_temp4])
df_temp3



RuntimeError: gene is not a column in the inputted data frame

In [27]:
bokeh.io.show(iqplot.stripbox(df_temp3,q="log2(intensity)", cats=["POI","strainType"],q_axis="y", jitter = True, width=800, height=800, title="Negative Deletions"))

In [12]:
df_res_pos = get_proteome_hits(df, map,genes_positive)
df_res_neg = get_proteome_hits(df, map,genes_negative)


0.0
0.00034223134839151266
0.0006844626967830253
0.001026694045174538
0.0013689253935660506
0.0017111567419575632
0.002053388090349076
0.0023956194387405884
0.0027378507871321013
0.003080082135523614
0.0034223134839151265
0.0037645448323066393
0.004106776180698152
0.004449007529089665
0.004791238877481177
0.00513347022587269
0.0054757015742642025
0.005817932922655715
0.006160164271047228
0.006502395619438741
0.006844626967830253
0.007186858316221766
0.007529089664613279
0.00787132101300479
0.008213552361396304
0.008555783709787816
0.00889801505817933
0.009240246406570842
0.009582477754962354
0.009924709103353867
0.01026694045174538
0.010609171800136893
0.010951403148528405
0.011293634496919919
0.01163586584531143
0.011978097193702943
0.012320328542094456
0.012662559890485968
0.013004791238877482
0.013347022587268994
0.013689253935660506
0.01403148528405202
0.014373716632443531
0.014715947980835045
0.015058179329226557
0.015400410677618069
0.01574264202600958
0.016084873374401096
0.0164

In [13]:
df_res_neg.head(50)

,proteinGroup,p-value,t_stat,gene,role
0,SPCC594.01,1.188486e-10,-6.590172,duc1,NaN
1,SPAC9E9.09c,4.054161e-10,6.388595,atd1,NaN
2,SPBC8D2.10c,2.261869e-07,-5.254445,rmt3,NaN
3,SPCC13B11.01,4.470600e-06,4.642939,adh1,NaN
4,SPBC18E5.06,9.314676e-06,-4.481972,rps21,NaN
5,SPBC8D2.06,9.574956e-06,4.475834,irs1,NaN
6,SPBC16E9.15,1.531059e-05,-4.370198,SPBC16E9.15,NaN
7,SPAC22G7.06c,1.629587e-05,4.356001,ura1,NaN
8,SPAC1B2.03c,1.900164e-05,4.320867,elo2,NaN
9,SPBC83.11,2.101307e-05,4.297724,pet2,NaN


In [14]:
df_res_pos.head(50)

,proteinGroup,p-value,t_stat,gene,role
0,SPCC550.15c,1.536889e-13,-7.612058,rei1,NaN
1,SPBC19C7.04c,4.205589e-12,-7.118594,SPBC19C7.04c,NaN
2,SPAC1142.07c,1.787947e-11,-6.894417,vps32,NaN
3,SPBC3B9.06c,4.155218e-10,6.386307,atg3,NaN
4,SPBPB2B2.12c,4.289278e-10,6.381017,gal10,NaN
5,SPCPB16A4.04c,1.009394e-09,6.237094,trm8,NaN
6,SPBC405.05,2.209992e-09,6.102944,atg16,NaN
7,SPBC17D1.02,2.087993e-08,-5.704567,dph2,NaN
8,SPBC32F12.08c,1.275293e-07,5.366294,duo1,NaN
9,SPAC6F12.13c,1.698503e-07,-5.311110,fps1,NaN


In [12]:
#merge the heads of the two dataframes on gene
df_res_head_merged = pd.merge(df_res_pos.head(200), df_res_neg.head(200), on='gene', how='inner')
df_res_head_merged.head(50)

,proteinGroup_x,p-value_x,t_stat_x,gene,role_x,proteinGroup_y,p-value_y,t_stat_y,role_y
0,SPBPB2B2.12c,4.289278e-10,6.381017,gal10,NaN,SPBPB2B2.12c,3.655742e-03,2.921202,NaN
1,SPAC1782.03,8.627940e-03,2.637750,saf3,NaN,SPAC1782.03,1.858461e-02,2.362029,NaN
2,SPAC9E9.09c,1.027372e-02,2.577078,atd1,NaN,SPAC9E9.09c,4.054161e-10,6.388595,NaN
3,SPBC215.08c,2.714524e-02,-2.216492,arg4,NaN,SPBC215.08c,6.450489e-03,2.736287,NaN
4,SPBC557.03c,4.999939e-02,1.965128,pim1,NaN,SPBC557.03c,7.918553e-03,2.667016,NaN
5,SPAC4F10.17,5.167965e-02,1.950856,SPAC4F10.17,NaN,SPAC4F10.17,7.687356e-03,2.677114,NaN
6,SPAC3H1.07,5.778908e-02,1.902039,aru1,NaN,SPAC3H1.07,1.068747e-02,2.563066,NaN
7,SPBC31A8.01c,6.496591e-02,-1.849912,rtn1,NaN,SPBC31A8.01c,1.294449e-04,-3.859820,NaN
8,SPAC521.03,7.810812e-02,1.765700,SPAC521.03,NaN,SPAC521.03,4.324385e-05,4.128556,NaN
9,SPBC18E5.04;SPAP7G5.05,9.017427e-02,1.698044,SPBC18E5.04;SPAP7G5.05,NaN,SPBC18E5.04;SPAP7G5.05,3.070591e-04,3.636556,NaN


In [9]:
#does df_res_neg have pfl3 as a gene
df_res_neg.loc[df_res_neg['gene'] == 'SPBC359.04c']

,proteinGroup,p-value,t_stat,gene,role


In [10]:
#find SPAC1F8.06 in the df_temp and df_temp2 and plot the log2(intensity) for each with matplotlib
import matplotlib.pyplot as plt
#extract numpy arrays
df_temp3 = df_temp.loc[df_temp['proteinGroup'] == 'SPAC2F3.16']
df_temp4 = df_temp2.loc[df_temp2['proteinGroup'] == 'SPAC2F3.16']
#do t-test and print results
ttest = st.ttest_ind(df_temp3['log2(intensity)'], df_temp4['log2(intensity)'])
print(ttest)
#get means
mean1 = df_temp3['log2(intensity)'].mean()
mean2 = df_temp4['log2(intensity)'].mean()
#print them
print(mean1)
print(mean2)


NameError: name 'df_temp' is not defined